# Detection of Fake / Real Profiles

## Neural Network

Import libraries

In [1]:
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import pickle 

import pandas_profiling as pp
from pandas_profiling import ProfileReport
import keras as k
from sklearn.model_selection import train_test_split
import keras
from keras import regularizers
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from numpy.random import seed
seed(1)

Import Datasets

In [2]:

df_users = pd.read_csv("F:/New folder/Fake_Profile_Detection_using_ML_and_dd/data/users.csv")
    #print("\nFake Dataset\n")
df_fusers = pd.read_csv("F:/New folder/Fake_Profile_Detection_using_ML_and_dd/data/Fusers.csv")

Displays the columns of fake users

In [3]:
df_fusers.shape

(1337, 34)

Display the columns of Real Users

In [4]:
df_users.shape

(1481, 34)

Assign the each values to the varible based on Status

In [5]:
#for dataframe_users
isNotFake = np.zeros(1481)

#for dataframe_fusers
isFake = np.ones(1337)

In [6]:
#adding is fake or not column to make predictions for it
df_fusers["isFake"] = isFake
df_users["isFake"] = isNotFake

In [7]:
df_allUsers = pd.concat([df_fusers, df_users], ignore_index=True)
df_allUsers.columns = df_allUsers.columns.str.strip()

In [8]:
#to shuffle the whole data
df_allUsers = df_allUsers.sample(frac=1).reset_index(drop=True)

In [9]:
df_allUsers.describe()

,id,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,geo_enabled,profile_use_background_image,profile_background_tile,utc_offset,protected,verified,isFake
count,2.818000e+03,2818.000000,2818.000000,2818.000000,2818.000000,2818.000000,1728.0,8.0,721.0,2760.0,489.0,1069.000000,0.0,0.0,2818.000000
mean,5.374889e+08,1672.198368,371.105039,395.363023,234.541164,2.818666,1.0,1.0,1.0,1.0,1.0,1478.391020,NaN,NaN,0.474450
std,2.977005e+08,4884.669157,8022.631339,465.694322,1445.847248,23.480430,0.0,0.0,0.0,0.0,0.0,8108.211889,NaN,NaN,0.499435
min,3.610511e+06,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,-39600.000000,NaN,NaN,0.000000
25%,3.620867e+08,35.000000,17.000000,168.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,0.000000
50%,6.162253e+08,77.000000,26.000000,306.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,0.000000
75%,6.177673e+08,1087.750000,111.000000,519.000000,37.000000,1.000000,1.0,1.0,1.0,1.0,1.0,3600.000000,NaN,NaN,1.000000
max,1.391998e+09,79876.000000,408372.000000,12773.000000,44349.000000,744.000000,1.0,1.0,1.0,1.0,1.0,36000.000000,NaN,NaN,1.000000


In [10]:
df_allUsers.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset,isFake
0,617315471,Sherlene William,sherlenewill,59,22,540,0,0,Sun Jun 24 17:38:26 +0000 2012,NaN,...,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,My name is satyam . I am experienced in Wep Ap...,2015-02-14 10:40:01,INT,1.0
1,331312863,ILARIA,ilaria_33,998,328,1435,58,0,Fri Jul 08 00:08:42 +0000 2011,NaN,...,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,"Certa di pochissime cose, sicura di tanti dubb...",2015-02-14 10:54:49,E13,0.0
2,299837277,Andrea Latino,andrealatino,3866,533,51,0,15,Mon May 16 19:18:30 +0000 2011,http://t.co/btA2R4ZdVq,...,http://a0.twimg.com/images/themes/theme14/bg.gif,EEEEEE,BC252C,3600.0,NaN,NaN,22. Global Shaper @WEF. Speaker @TEDxBathUni. ...,2015-02-14 10:54:49,E13,0.0
3,148536610,Maurizio Mucciola,maurizio_mwg,1063,88,228,35,1,Wed May 26 23:40:12 +0000 2010,http://www.flickr.com/photos/maurizio_mwg/sets/,...,http://a0.twimg.com/profile_background_images/...,EDECE9,BA2800,0.0,NaN,NaN,Architect at Kengo Kuma & Associates,2015-02-14 10:54:49,E13,0.0
4,420995289,raffaele sarno,SarnoRaffaele,15324,301,2001,15749,1,Fri Nov 25 11:00:19 +0000 2011,NaN,...,http://a0.twimg.com/images/themes/theme7/bg.gif,EBEBEB,990000,3600.0,NaN,NaN,sono un ragazzo semplice un saluto ciao,2015-02-14 10:54:49,E13,0.0


In [11]:
Y = df_allUsers.isFake

In [12]:
df_allUsers.drop(["isFake"], axis=1, inplace=True)
X = df_allUsers

In [13]:
Y.reset_index(drop=True, inplace=True)

In [14]:
print(Y.shape)

(2818,)


In [15]:
X.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,617315471,Sherlene William,sherlenewill,59,22,540,0,0,Sun Jun 24 17:38:26 +0000 2012,NaN,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,My name is satyam . I am experienced in Wep Ap...,2015-02-14 10:40:01,INT
1,331312863,ILARIA,ilaria_33,998,328,1435,58,0,Fri Jul 08 00:08:42 +0000 2011,NaN,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,"Certa di pochissime cose, sicura di tanti dubb...",2015-02-14 10:54:49,E13
2,299837277,Andrea Latino,andrealatino,3866,533,51,0,15,Mon May 16 19:18:30 +0000 2011,http://t.co/btA2R4ZdVq,...,EFEFEF,http://a0.twimg.com/images/themes/theme14/bg.gif,EEEEEE,BC252C,3600.0,NaN,NaN,22. Global Shaper @WEF. Speaker @TEDxBathUni. ...,2015-02-14 10:54:49,E13
3,148536610,Maurizio Mucciola,maurizio_mwg,1063,88,228,35,1,Wed May 26 23:40:12 +0000 2010,http://www.flickr.com/photos/maurizio_mwg/sets/,...,E3E2DE,http://a0.twimg.com/profile_background_images/...,EDECE9,BA2800,0.0,NaN,NaN,Architect at Kengo Kuma & Associates,2015-02-14 10:54:49,E13
4,420995289,raffaele sarno,SarnoRaffaele,15324,301,2001,15749,1,Fri Nov 25 11:00:19 +0000 2011,NaN,...,F3F3F3,http://a0.twimg.com/images/themes/theme7/bg.gif,EBEBEB,990000,3600.0,NaN,NaN,sono un ragazzo semplice un saluto ciao,2015-02-14 10:54:49,E13


In [16]:
lang_list = list(enumerate(np.unique(X["lang"])))
lang_dict = {name : i for i, name in lang_list}
X.loc[:, "lang_num"] = X["lang"].map(lambda x: lang_dict[x]).astype(int)

X.drop(["name"], axis=1, inplace=True)

In [17]:
X = X[[
    "statuses_count",
    "followers_count",
    "friends_count",
    "favourites_count",
    "lang_num",
    "listed_count",
    "geo_enabled",
    "profile_use_background_image"
                        ]]

In [18]:
profile = ProfileReport(X, title="Pandas Profiling Report")


In [19]:
X = X.replace(np.nan, 0) #To replace the missing boolean values with zeros as it means false

In [20]:
profile = ProfileReport(X, title="Pandas Profiling Report")


In [21]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size=0.8, test_size=0.2, random_state=0)

In [22]:
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(1803, 8)
(564, 8)
(1803,)
(564,)


In [23]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(64, input_dim=32, activation='relu'))
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(32,input_dim=64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8,673
Trainable params: 8,673
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
history = model.fit(train_X, train_y,
                    epochs=15,
                    verbose=1,
                    validation_data=(val_X,val_y))

Epoch 1/15
57/57 [==============================] - 4s 18ms/step - loss: 9.7833 - accuracy: 0.8708 - val_loss: 0.6173 - val_accuracy: 0.9845
Epoch 2/15
57/57 [==============================] - 1s 11ms/step - loss: 1.1488 - accuracy: 0.9861 - val_loss: 0.3706 - val_accuracy: 0.9889
Epoch 3/15
57/57 [==============================] - 0s 9ms/step - loss: 0.7010 - accuracy: 0.9872 - val_loss: 0.1944 - val_accuracy: 0.9911
Epoch 4/15
57/57 [==============================] - 0s 7ms/step - loss: 0.3980 - accuracy: 0.9861 - val_loss: 0.0590 - val_accuracy: 0.9889
Epoch 5/15
57/57 [==============================] - 0s 5ms/step - loss: 0.1603 - accuracy: 0.9856 - val_loss: 0.0375 - val_accuracy: 0.9867
Epoch 6/15
57/57 [==============================] - 0s 4ms/step - loss: 0.9422 - accuracy: 0.9767 - val_loss: 0.2030 - val_accuracy: 0.9867
Epoch 7/15
57/57 [==============================] - 0s 2ms/step - loss: 0.3058 - accuracy: 0.9889 - val_loss: 0.1116 - val_accuracy: 0.9778
Epoch 8/15
57/57 [

In [26]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]*100)

file1 = open("resltnn.txt","w")#write mode
file1.write(str(score[1]*100))
file1.close()

Test loss: 1.7512903213500977
Test accuracy: 98.758864402771


In [27]:
# Plot training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
axes = plt.gca()
axes.set_xlim([0,14])
axes.set_ylim([0.85,1])
axes.grid(True, which='both')
axes.axhline(y=0.85, color='k')
axes.axvline(x=0, color='k')
axes.axvline(x=14, color='k')
axes.axhline(y=1, color='k')
plt.legend(['Train','Val'], loc='lower right')
plt.show()

C:\Users\MCA\AppData\Local\Temp\ipykernel_3472\2318103684.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
# Plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
axes = plt.gca()
axes.set_xlim([0,14])
axes.set_ylim([0,5])
axes.grid(True, which='both')
axes.axhline(y=0, color='k')
axes.axvline(x=0, color='k')
axes.axhline(y=5, color='k')
axes.axvline(x=14, color='k')
plt.legend(['Train','Val'], loc='upper right')
plt.show()

C:\Users\MCA\AppData\Local\Temp\ipykernel_3472\2592424667.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [29]:
# Write the index of the test sample to test
prediction = model.predict(test_X[136:137])
prediction = prediction[0]

print('\nThresholded output\n',(prediction>0.5)*1)


Thresholded output
 [1]


In [30]:
print(test_y[136:137])

953    1.0
Name: isFake, dtype: float64


In [31]:
# serialize model to JSON
model_json = model.to_json()

# Write the file name of the model

with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
# Write the file name of the weights

model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [32]:
# load json and create model

# Write the file name of the model

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
# Write the file name of the weights

loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [33]:
# create an iterator object with write permission - model.pkl
with open('model_picklenn.pkl', 'wb') as f:
    # save the model to disk
    model_pickle = 'model_picklenn.pkl'
    pickle.dump(model_pickle, f)